In [ ]:
!conda install --yes -c conda-forge geopandas
!pip uninstall pyproj
!pip install pyproj


In [1]:
import pandas as pd
from scripts import gee_satellite_data
from scripts import gee_functions
from scripts import gis_functions
from scripts import general_functions

In [2]:
import ee
ee.Initialize()

## Download Sentinel 1 data

In [3]:
querydata = gee_satellite_data.get_gee_data("2016-02-01",
                                            "2016-09-30", 
                                            "data/col_t3.shp", 
                                            "sentinel1", bands = ['VV'])

In [4]:
querydata.summary

,dates,cover_percentage
0,2016-03-26 18:13:23,99.558320
1,2016-05-13 18:13:30,99.559609
2,2016-09-10 18:13:37,99.556874
3,2016-02-07 18:13:28,99.559613
4,2016-02-14 05:42:59,99.558320
5,2016-03-02 18:13:28,99.558167
6,2016-03-09 05:42:58,99.556873
7,2016-04-02 05:43:00,99.558320
8,2016-04-19 18:13:29,99.561045
9,2016-04-26 05:43:00,99.558310


Due to Sentinel 1 mission has two satellites (a and b), it can orbit the earth in a different orientation (Ascending and Descending), the  6 days data can be summarised averaging over a time window. 

In [5]:
querydata.reduce_by_days(10)
querydata._dates_reduced

0    2016-03-29 23:58:12
1    2016-04-19 18:13:29
2    2016-04-26 05:43:00
3    2016-05-13 18:13:30
4    2016-05-20 05:43:02
5    2016-06-09 23:58:18
6    2016-06-30 18:13:33
7    2016-07-07 05:43:05
8    2016-07-27 23:58:20
9    2016-08-17 18:13:36
10   2016-08-24 05:43:07
11   2016-09-10 18:13:37
dtype: datetime64[ns]

In [6]:
imagetoplot = querydata.image_collection.toList(querydata.image_collection.size()).get(2)

radarParams = {'gamma': 1.3, 
                   'min': -25,
                   'max': 5,
                   'bands': ['VV']
                   }

gee_satellite_data.plot_eeimage(imagetoplot, radarParams, querydata.geometry, zoom = 11)


if the time window reduction step was completed, the code will download all the images that were calculated in that step. Otherwise, it will download the images that were acquired from the inital query

In [11]:
gee_satellite_data.download_gee_tolocal(querydata, 'gee_satellitedata/s1_processed', "col_t3", 10)

-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160329_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160419_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160426_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160513_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160520_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160609_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160630_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160707_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160727_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_20160817_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s1_processed\s1_grd_201608

## Download Landsat 8 Surface Reflectance data

In [7]:
landsat8 = gee_satellite_data.get_gee_data("2018-01-01",
                                           "2018-12-31", 
                                           "data/col_t3.shp", 
                                           "landsat8_t1sr", 
                                            bands = ['B2', 'B3', 'B4', 'B5', 'B10', 'B11'],
                                            cloud_percentage= 70)

In [8]:
landsat8.summary

,dates,cover_percentage
0,2018-02-13 00:00:00,2.529917
1,2018-03-01 10:12:27,46.204227
2,2018-03-17 00:00:00,92.968971
3,2018-08-08 10:12:07,99.550840
4,2018-08-24 00:00:00,62.864823
5,2018-09-09 00:00:00,97.086428
6,2018-11-28 10:13:04,0.000000
7,2018-12-30 00:00:00,88.610031


In [10]:

truecolorParams = {'gamma': 1.3, 
                   'min': 57,
                   'max': 2000,
                   'bands': ['B4','B3','B2']
                   }

imagetoplot = landsat8.image_collection.toList(landsat8.image_collection.size()).get(5)

gee_satellite_data.plot_eeimage(imagetoplot, truecolorParams, landsat8.geometry, zoom = 11)



### Adding vegetation index layers

In [11]:
landsat8.add_vi_layer("ndvi")

In [13]:
ndviParams = {     'min': 0,
                   'max': 1,
                   'palette': ['#FF0000', '#00FF00'],
                   'bands': ['ndvi']
                   }

imagetoplot = landsat8.image_collection.toList(landsat8.image_collection.size()).get(5)

gee_satellite_data.plot_eeimage(imagetoplot, ndviParams, landsat8.geometry, zoom = 11)

### Registrate Landsat 8 images using a sentinel surface reflectance image

In [14]:
displacement, s2image, l8image  = landsat8.l8_displacement()


the S2 image reference was found in 2019-06-07


In [15]:
landsat8.displace_landsatcollection(displacement)

the image collection was resgistered


In [16]:
import folium
import geehydro

registered = l8image.displace(displacement);

truecolorParams = {'gamma': 1.3, 
                   'min': 57,
                   'max': 2000,
                   'bands': ['B4','B3','B2']
                   }

centergeometry = gis_functions.geometry_center(landsat8.geometry)


Map = folium.Map(location=[centergeometry[1],
                           centergeometry[0]], zoom_start=11)

eegeom = gis_functions.polygon_fromgeometry(landsat8.geometry)
eegeom = gee_functions.geometry_as_ee(eegeom)

Map.addLayer(ee.Image(s2image), truecolorParams, 'gee image s2')
Map.addLayer(ee.Image(l8image), truecolorParams, 'gee image ls')
Map.addLayer(ee.Image(registered), truecolorParams, 'gee image ls registered')


Map.addLayer(ee.Image().paint(eegeom, 1, 3), {}, 'region of interest:')


Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

Map

### Download data

In [10]:
gee_satellite_data.download_gee_tolocal(landsat8, ## gee_data class
                                        'gee_satellitedata/l8_processed', ## output directory
                                        "col_t3", ## suffix identifiacation
                                        10, # spatial resolutions
                                        bands = ['B2', 'B3', 'B4', 'B5', 'B10', 'B11', 'ndvi'], ## bands to be exported
                                        cover_percentage= 75 ## minimum cover percentage
                                       )

-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20160327_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20160530_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20160717_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20160903_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20161021_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20161106_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20161122_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20170314_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20170602_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20170704_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_

## Download Sentinel 2 Surface Reflectance data

In [11]:
sentinel2 = gee_satellite_data.get_gee_data("2019-06-01", ## start date
                                            "2019-09-28", ## end query date
                                            "data/col_t3.shp", ## region of interest
                                            "sentinel2_sr", ## mission
                                            cloud_percentage= 80 ## cloud percetange per image 
                                           )

In [12]:
sentinel2.summary

,dates,cover_percentage
0,2019-06-08 00:00:00,98.297849
1,2019-06-13 10:32:02,19.416454
2,2019-07-03 00:00:00,79.603491
3,2019-07-13 00:00:00,53.544290
4,2019-07-23 00:00:00,0.000000
5,2019-08-02 00:00:00,78.819392
6,2019-08-07 00:00:00,72.383079
7,2019-08-12 00:00:00,99.529571
8,2019-08-17 00:00:00,52.523547
9,2019-08-22 00:00:00,99.110287


In [13]:
truecolorParams = {'gamma': 1.3, 
                   'min': 57,
                   'max': 2000,
                   'bands': ['B4','B3','B2']
                   }

imagetoplot = sentinel2.image_collection.toList(sentinel2.image_collection.size()).get(9)

gee_satellite_data.plot_eeimage(imagetoplot, truecolorParams, sentinel2.geometry, zoom = 11)

### Adding vegetation index layers

In [14]:
sentinel2.add_vi_layer("ndvi")

In [15]:
ndviParams = {     'min': 0,
                   'max': 1,
                   'palette': ['#FF0000', '#00FF00'],
                   'bands': ['ndvi']
                   }

imagetoplot = sentinel2.image_collection.toList(sentinel2.image_collection.size()).get(9)

gee_satellite_data.plot_eeimage(imagetoplot, ndviParams, sentinel2.geometry, zoom = 11)

### Download data

In [17]:
gee_satellite_data.download_gee_tolocal(sentinel2, ## 
                                        'gee_satellitedata/s2_processed', ## outputpath 
                                        "col_t3", ## a reference for the area that was query
                                        10, ## the spatial resolution in meters
                                        bands = ['B2', 'B3', 'B4', 'B8', 'ndvi']
                                       )

-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190608_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190613_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190703_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190713_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190723_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190802_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190807_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190812_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190817_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190822_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_201908